In [25]:
import requests 
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
def links(urls, x):
    """
    Function to get website urls
    Takes in a url from any website 
    and a filter by to search for what urls start with
    
    """
    new_list = []
    content = requests.get(urls)

    soup = BeautifulSoup(content.text, 'html.parser')
    href = soup.find_all('a', href = True)
    for link in href:
        if link['href'].startswith(x):
            new_list.append(link['href'])
    
    return new_list

In [27]:
def players_links(url, x, limit):
    """
    import bs4
    import request
    Gets back a list of url 
    argument url takes a actual url
    x filters out href elements
    Created by Jesus Christ
    
    """
    players_li = []
    movie_url = url
    req_url = requests.get(movie_url)
    soup = BeautifulSoup(req_url.text, 'html.parser')
    href = soup.find_all('a', href = True)
    for links in href: 
        if links['href'].startswith(x):
            players_li.append(links['href'])
    return(players_li[0:limit])


In [31]:
stats_page = 'https://www.hltv.org/stats/players?startDate=all'

In [32]:
players_url = players_links(stats_page, '/stats/players/', 696)

In [33]:
len(players_url)

0

In [20]:
players_url

[]

In [8]:
list_17 = []
for link in players_url:
    
    home_url = "https://www.hltv.org/" + link
    
    req_url = requests.get(home_url)
        
    soup = BeautifulSoup(req_url.text)
    
    player_dict = {}
    
    player_dict['Name'] = soup.find('h1').text
    
    table = soup.find('div', class_='summaryInfoContainer')
    player_dict['Team'] = table.find('div', class_='SummaryTeamname').text  
    
    data = soup.find_all('span')
    
    for i, span in enumerate(data):
        if 'Total kills' in span:
            player_dict['total_kills'] = data[i+1].text
        if 'Headshot %' in span:
            player_dict['headshot_%'] = data[i+1].text
        if 'Total deaths' in span:
            player_dict['total_deaths'] = data[i+1].text
        if 'K/D Ratio' in span:
            player_dict['k/d_ratio'] = data[i+1].text
        if 'Damage / Round' in span:
            player_dict['damage/round'] = data[i+1].text
        if 'Grenade dmg / Round' in span:
            player_dict['grenadedmg/round'] = data[i+1].text
        if 'Maps played' in span:
            player_dict['maps_played'] = data[i+1].text
        if 'Rounds played' in span:
            player_dict['rounds_played'] = data[i+1].text
        if 'Kills / round' in span:
            player_dict['kills/round'] = data[i+1].text
        if 'Assists / round' in span:
            player_dict['assists/round'] = data[i+1].text
        if 'Deaths / round' in span:
            player_dict['death/round'] = data[i+1].text
        if 'Saved by teammate / round' in span:
            player_dict['savedbyteammate/round'] = data[i+1].text
        if 'Saved teammates / round' in span:
            player_dict['savedteammates/round'] = data[i+1].text
        
    
    list_17.append(player_dict)

In [9]:
list_17

[]

In [10]:
df = pd.DataFrame(list_17)

In [11]:
df.describe()

ValueError: Cannot describe a DataFrame without columns

In [ ]:
df['headshot_%'] = (df['headshot_%'].str.split('%').str[0].astype(np.float)/100)

In [ ]:
df

In [ ]:
df.columns.tolist()[1:]

In [ ]:
col_list = df.columns.tolist()[1:]
col_list.remove('Name')
col_list.remove('Team')
for c in col_list:
    df[c] = df[c].astype(float)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
team_5 = pd.DataFrame((df['Team'].value_counts() == 5).astype(int))

In [ ]:
len(team_5[team_5['Team'] == 231])

In [ ]:
df['Team'].value_counts().unique()

In [ ]:
len(df['Team'].unique())

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
plt.scatter(df['headshot_%'], df['total_kills']);

In [ ]:
km = KMeans(n_clusters=3)
km

In [ ]:
y_preds = km.fit_predict(df[['headshot_%', 'total_kills']])
y_preds

In [ ]:
df['cluster'] = y_preds
df.head()

In [ ]:
df1 = df[df.cluster==0]
df2 = df[df.cluster==1]
df3 = df[df.cluster==2]

plt.scatter(df1['headshot_%'], df1.total_kills, color='green')
plt.scatter(df2['headshot_%'], df2.total_kills, color='red')
plt.scatter(df3['headshot_%'], df3.total_kills, color='blue')

plt.xlabel('HeadShot %')
plt.ylabel('Total Kills')
;